In [6]:
import pandas as pd

In [16]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
## Read the sample order files showing all purchases


In [19]:
orders = pd.read_csv('sample-orders.csv', sep = ',')

In [20]:
orders.head()

,order_date,order_id,customer,grand_total
0,9/7/11,CA-2011-100006,Dennis Kane,378
1,7/8/11,CA-2011-100090,Ed Braxton,699
2,3/14/11,CA-2011-100293,Neil Franz�sisch,91
3,1/29/11,CA-2011-100328,Jasper Cacioppo,4
4,4/8/11,CA-2011-100363,Jim Mitchum,21


In [21]:
##Create the RFM Table

In [22]:
#Recency is calculated for a point in time and the last order is Dec 31 2014, we will use that date for calcuations


In [23]:
import datetime as dt
NOW = dt.datetime(2014,12,31)

In [24]:
# Create the date of order as datetime

In [25]:
orders['order_date'] = pd.to_datetime(orders['order_date'])

In [26]:
orders.head()

,order_date,order_id,customer,grand_total
0,2011-09-07,CA-2011-100006,Dennis Kane,378
1,2011-07-08,CA-2011-100090,Ed Braxton,699
2,2011-03-14,CA-2011-100293,Neil Franz�sisch,91
3,2011-01-29,CA-2011-100328,Jasper Cacioppo,4
4,2011-04-08,CA-2011-100363,Jim Mitchum,21


In [27]:
## Create the RFM Table

In [31]:
rfmTable = orders.groupby('customer').agg({'order_date': lambda x: (NOW - x.max()).days, #Recency
                                          'order_id' : lambda x: len(x), #Frequency
                                           'grand_total' : lambda x: x.sum()}) #Monetary Value



In [32]:
rfmTable['order_date'] = rfmTable['order_date'].astype(int)
rfmTable.rename(columns= {'order_date' : 'recency',
                          'order_id' : 'frequency',
                          'grand_total' : 'monetary_value'}, inplace=True)

In [35]:
rfmTable.head()

,recency,frequency,monetary_value
customer,,,
Aaron Bergman,415,3,887
Aaron Hawkins,12,7,1744
Aaron Smayling,88,7,3050
Adam Bellavance,54,8,7756
Adam Hart,34,10,3249


In [37]:
## Customer Aaron Bergman has recency 415 days, frequency 3, monetary value $887 

In [39]:
aaron = orders[orders['customer']=='Aaron Bergman']

In [40]:
aaron

,order_date,order_id,customer,grand_total
624,2011-02-19,CA-2011-152905,Aaron Bergman,13
665,2011-03-07,CA-2011-156587,Aaron Bergman,310
2336,2013-11-11,CA-2013-140935,Aaron Bergman,564


In [41]:
## Confirm date of Aaron purchase and compare to the recency in the rfmTable

In [42]:
(NOW - dt.datetime(2013,11,11)).days == 415

True

In [43]:
## Determining RFM Quartiles

In [45]:
quantiles = rfmTable.quantile(q = [0.25,0.5,0.75])

In [46]:
quantiles

,recency,frequency,monetary_value
0.25,30.0,5.0,1145.0
0.50,75.0,6.0,2257.0
0.75,183.0,8.0,3784.0


In [47]:
## Exporting quantiles to dictionary for easier use

In [48]:
quantiles = quantiles.to_dict()

In [49]:
quantiles

{'recency': {0.25: 30.0, 0.5: 75.0, 0.75: 183.0},
 'frequency': {0.25: 5.0, 0.5: 6.0, 0.75: 8.0},
 'monetary_value': {0.25: 1145.0, 0.5: 2257.0, 0.75: 3784.0}}

In [50]:
## Creating the RFM Segementation Table

In [51]:
rfmSegmentation = rfmTable

In [52]:
## Two classes for teh RFM Segmentation since high recency is bad, while high frequency and monetary value is good 

In [64]:
# Arguments ( x = value, p = recency, monetary_value, frequency, k = quantiles dict)
def RClass(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]:
        return 3
    else:
        return 4
    

# Arguments ( x = value, p = recency, monetary_value, frequency, k = quantiles dict)
def FMClass(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]:
        return 2
    else:
        return 1
    

In [65]:
rfmSegmentation['R_Quartile'] = rfmSegmentation['recency'].apply(RClass, args=('recency', quantiles,))
rfmSegmentation['F_Quartile'] = rfmSegmentation['frequency'].apply(FMClass, args=('frequency', quantiles,))
rfmSegmentation['M_Quartile'] = rfmSegmentation['monetary_value'].apply(FMClass, args = ('monetary_value', quantiles))


In [66]:
rfmSegmentation['RFMClass'] = rfmSegmentation.R_Quartile.map(str) \
                            + rfmSegmentation.F_Quartile.map(str) \
                            + rfmSegmentation.M_Quartile.map(str) 

In [67]:
rfmSegmentation.head()

,recency,frequency,monetary_value,R_Quartile,F_Quartile,M_Quartile,RFMClass
customer,,,,,,,
Aaron Bergman,415,3,887,4,4,4,444
Aaron Hawkins,12,7,1744,1,2,3,123
Aaron Smayling,88,7,3050,3,2,2,322
Adam Bellavance,54,8,7756,2,2,1,221
Adam Hart,34,10,3249,2,1,2,212


In [68]:
rfmSegmentation.to_clipboard()

In [69]:
rfmSegmentation.to_csv('rfm-table.csv', sep=',')

In [70]:
## Top 5 best customers? by RFM Class(111), high spender who buy recently and frequently?

In [80]:
rfmSegmentation[rfmSegmentation['RFMClass']=='111'].sort_values('monetary_value', ascending = False).head(5)

,recency,frequency,monetary_value,R_Quartile,F_Quartile,M_Quartile,RFMClass
customer,,,,,,,
Sanjit Engle,9,11,12210,1,1,1,111
John Lee,21,11,9801,1,1,1,111
Pete Kriz,9,12,8647,1,1,1,111
Harry Marie,2,10,8237,1,1,1,111
Lena Creighton,16,12,7661,1,1,1,111
